## Problems 1 and 2

### Problem 1
Write a class called Signal for storing digital audio signals. The constructor should accept a sample rate (an integer) and an array of samples (a NumPy array). Store these inputs as attributes.
Write a method called plot() that generates the graph of the sound wave. Use the sample rate to label the x-axis in terms of seconds. See Figure 9.1 for an example.

### Problem 2
Add a method to the Signal class called export() that accepts a file name and generates a .wav file from the sample rate and the array of samples. Scale the array of samples appropriately before writing to the output file. Ensure that your scaling technique is valid for arbitrary arrays of samples. Note that some arrays will not need to be scaled.

In [ ]:
from matplotlib import pyplot as plt
from scipy.io import wavfile as wf
import math
import scipy
import numpy as np
%matplotlib inline

class Signal(object) :
    def __init__(self,rate,samples) :
        """
        Input : rate (int) for how often we sampled
                samples (numpy array) samples
        """
        self.rate = int(rate)
        self.samples = samples


    def plot(self,sound_name='Your Sound') :
        """
        Plots the sound wave described by the samples
        """
        n = self.samples.shape[0]
        x = np.linspace(0,float(n)/self.rate,n)
        plt.plot(x,self.samples,'b')
        plt.xlabel('Seconds')
        plt.ylabel('Frequency')
        plt.title('Sound Wave')
        plt.show()

## Problem 3
The ‘A’ note occurs at a frequency of 440 Hertz. Generate the sine wave that corresponds to an ‘A’ note being played for 5 seconds.
Once you have successfully generated the ‘A’ note, experiment with different frequencies to generate different notes.

In [ ]:
samplerate = 44100
frequency = 440 #A
#frequency = 493.88 #B
#frequency = 523.25 #C
#frequency = 587.33 #D
#frequency = 659.25 #E
#frequency = 698.46 #F
#frequency = 783.99 #G
length = 5
wave_function = lambda x : np.sin(2*np.pi*x*frequency)
stepsize = 1./samplerate
sample_points = np.arange(0,length,stepsize)
samples = wave_function(sample_points)
sinewave = Signal(samplerate,samples)
sinewave.export("sine.wav")

## Problem 4
Write a function that accepts a NumPy array and computes the discrete Fourier transform of the array using Equation 9.1. Return the array of calculated coeficients.

In [ ]:
# Not working
def DFT(X) :
    shape = X.shape
    X = X.reshape(1,-1)
    A = list(X)
    N = len(A)
    dft = lambda x : sum([A[n]*math.exp((2*np.pi*A.index(x)*n/N)*1j) for n in xrange(N)])
    c = [dft(x) for x in A]
    return c

## Problem 5
Update the plot() method in the Signal class so that it gener- ates a single plot with two subplots: the original soundwave, and the mag- nitude of the coe cients of the DFT (as in Figure 9.3). Use one of SciPy’s FFT implementations to calculate the DFT.

In [1]:
from matplotlib import pyplot as plt
from scipy.io import wavfile as wf
import math
import scipy
import numpy as np

class Signal(object) :
    def __init__(self,rate,samples) :
        """
        Input : rate (int) for how often we sampled
                samples (numpy array) samples
        """
        self.rate = int(rate)
        self.samples = samples


    def plot(self,sound_name='Your Sound') :
        """
        Plots the sound wave described by the samples
        """
        n = self.samples.shape[0]
        x = np.linspace(0,float(n)/self.rate,n)
        plt.subplot(1,2,1)
        plt.plot(x,self.samples,'b')
        plt.xlabel('Seconds')
        plt.ylabel('Frequency')
        plt.title('Sound Wave')
        
        dft = abs(scipy.fft(self.samples))
        x_vals = np.arange(1,len(dft)+1,1)*1.
        x_vals /= len(self.samples)
        x_vals *= self.rate

        plt.subplot(1,2,2)
        plt.plot(x_vals,dft,'b')
        plt.xlabel('Hz')
        plt.title('DFT')
        plt.axis([0,self.rate,0,max(dft)])
        plt.suptitle(sound_name)
        plt.show()

    def export(self,filename) :
        b = max(self.samples)
        a = min(self.samples)

        new_samples = self.samples*32767*2/(b-a)
        new_samples -= (b+a)/2
        new_samples = np.int16(new_samples)

        wf.write(filename,self.rate,new_samples)

    def __add__(self,other) :
        if self.rate != other.rate:
            raise ValueError("Sounds incompatible.")
        else :
            if self.samples.shape[0] > other.samples.shape[0] :
                needed = self.samples.shape[0]-other.samples.shape[0]
                zeros = np.zeros(shape=(1,needed))[0]
                padded_other = np.hstack((other.samples,zeros))
                new_signal = self.samples + padded_other
            elif self.samples.shape[0] < other.samples.shape[0] :
                needed = other.samples.shape[0]-self.samples.shape[0]
                zeros = np.zeros(shape=(1,needed))[0]
                padded_self = np.hstack((self.samples,zeros))
                new_signal = padded_self + other.samples
            else :
                new_signal = self.samples + other.samples
            return Signal(self.rate,new_signal)

## Problem 6
Create several chords and observe the plot of their DFT. There should be as many spikes as there are notes in the plot. Then create a sound that changes over time.

In [2]:
"""
Note : Problem 5 must be run before this block
"""
samplerate = 44100
length = 0.5
wave_function = lambda x,f : np.sin(2*np.pi*x*f)
stepsize = 1./samplerate
sample_points = np.arange(0,length,stepsize)
notes = {'A':440,'B':493.88,'C':523.25,'D':587.33,'E':659.25,'F':698.46,
                'G':783.99}
waves = {}
for a in notes.keys() :
    waves[a] = Signal(samplerate,wave_function(sample_points,notes[a]))

waves['HA'] = Signal(samplerate,wave_function(sample_points,2*notes['A']))
waves['LG'] = Signal(samplerate,wave_function(sample_points,0.5*notes['G']))

cmajor1 = waves['C'] + waves['E'] + waves['G']
cmajor2 = waves['C'] + waves['F'] + waves['HA']
cmajor3 = waves['B'] + waves['D'] + waves['G']
aminor1 = waves['A'] + waves['C'] + waves['E']
aminor2 = waves['A'] + waves['D'] + waves['F']
aminor3 = waves['LG'] + waves['B'] + waves['E']

aminor = np.hstack((waves['A'].samples,waves['B'].samples,waves['C'].samples,
            waves['D'].samples,waves['E'].samples, waves['F'].samples,
            waves['G'].samples,waves['HA'].samples,waves['G'].samples,
            waves['F'].samples,waves['E'].samples,waves['D'].samples,
            waves['C'].samples,waves['B'].samples,waves['A'].samples,
            aminor1.samples,aminor2.samples,aminor1.samples,
            aminor3.samples,aminor1.samples))

a_minor_scale = Signal(samplerate,aminor)
cmajor1.plot()
cmajor2.plot()
cmajor3.plot()
aminor1.plot()
aminor2.plot()
aminor3.plot()
a_minor_scale.plot()

#For the enjoyment of listening to an A-minor scale
a_minor_scale.export('a-minor_scale.wav')